In [ ]:
import re
# import wordninja as nj
# import nltk
from wordsegment import segment, load

In [ ]:
tipo_datos = ['boolean', 'char', 'byte', 'short', 'int', 'long', 'float', 'double', 'String', 'Array']

In [ ]:
def camel_case_split(identifier):
    matches = re.finditer(
        r".+?(?:(?<=[a-z])(?=[A-Z])|(?<=[A-Z])(?=[A-Z][a-z])|$)", identifier
    )
    
    return [m.group(0).strip() for m in matches]

In [ ]:
# from __future__ import division
# from collections import Counter
# import nltk

# WORDS = nltk.corpus.brown.words()
# COUNTS = Counter(WORDS)


# def pdist(counter):
#     N = sum(counter.values())
#     return lambda x: counter[x] / N


# P = pdist(COUNTS)


# def Pwords(words):
#     return product(P(w) for w in words)


# def product(nums):
#     result = 1
#     for x in nums:
#         result *= x
#     return result


# def splits(text, start=0, L=20):
#     return [(text[:i], text[i:]) for i in range(start, min(len(text), L) + 1)]


# def segment(text):
#     if not text:
#         return []
#     else:
#         candidates = ([first] + segment(rest) for (first, rest) in splits(text, 1))
#         return max(candidates, key=Pwords)


# concatenated_word = "helloworldhowareyou"
# # concatenated_word = 'holamundo'
# segmented_words = segment(concatenated_word)
# print(segmented_words)  # ['acquire', 'customer', 'data']

In [ ]:
def separate_words(text):
    load()
    t_list = segment(text)
    
    return ''.join([item.strip().title() for item in t_list])

In [ ]:
text = """
    package com.hmkcode;

import java.io.BufferedReader;
import java.io.FileNotFoundException;
import java.io.FileReader;
import java.io.IOException;

public class textfilereader {

	private BufferedReader buffer;
	private String currentLine = "";
	public textfilereader(){ 
		
	}
	
	public void open(String file){
		
		try {
			close();
			
			buffer = new BufferedReader(new FileReader(file));
			
		} catch (FileNotFoundException e1) {
			e1.printStackTrace();
	
		}
		
	}
	
	public void close(){
		
		try {
			if(buffer != null){
				buffer.close();
				buffer = null;
			}
		} catch (IOException e) {
			e.printStackTrace();
		}
		
	}
	
	public String readLine() throws Exception{
		if(buffer != null){
			currentLine = buffer.readLine();
			
			if(currentLine == null)
				close();
			
			return currentLine;
		}
		else
			throw new Exception("No file to read...");
	}
	
	public String getCurrent(){
		return this.currentLine;
	}
	
	public boolean isReadable(){
		return (buffer != null && this.currentLine != null);
	}
}
"""

In [ ]:
# text = ''

# with open(file='./codigo.txt', mode='r', encoding='utf-8') as file:
#     text += file.read()

In [ ]:
class_starter = r"(class)\s([^\{]+)\{"

In [ ]:
texto = re.findall(pattern=class_starter, string=text)#[0][1]

if bool(texto):
    identificador = texto[0][0].strip()
    
    t_t = text.split('\n')
    n = len(t_t)
    
    for i in range(n):
        if identificador in t_t[i]:
            sep_l = t_t[i].split()
            # print(sep_l)
            n_2 = len(sep_l)
            for j in range(n_2):
                # print(sep_l[j])
                if sep_l[j] != identificador and sep_l[j] != 'public':
                    sep_l[j] = separate_words(sep_l[j])
                    print(sep_l[j])
                    break
            sep_l = " ".join(sep_l)
            t_t[i] = sep_l

In [ ]:
funciones_static = r"(static\s){1,1}.{0,}(){1,1}"
funciones_private = r"(private\s){1,1}.{0,}(){1,1}"


if "private" in text:

    coincidencias = re.findall(pattern=funciones_private, string=text)

    if bool(coincidencias):
        # print("segundo if")
        c = coincidencias[0][0].strip()
        t_l = text.split("\n")
        for i in range(len(t_l)):
            # print(c,line)
            if c in t_l[i]:
                # print("antes del title", t_l[i].strip())
                sep = t_l[i].split()
                # print(sep.index(c))
                for j in range(len(sep)):
                    if sep[j] != c and sep[j] not in tipo_datos:
                        sep[j] = separate_words(sep[j])
                        break
                sep = " ".join(sep)
                t_l[i] = sep
                
text = '\n'.join(t_l)



In [ ]:
import requests

In [ ]:
# %pip install requests

In [3]:
import requests
import json

url = "http://127.0.0.1:8000/prueba_1"

input_data_for_model = {
    "text_of_code": """ 
    package com.hmkcode;

import java.io.BufferedReader;
import java.io.FileNotFoundException;
import java.io.FileReader;
import java.io.IOException;

public class textfilereader {

	private BufferedReader buffer;
	private String currentLine = "";
	public textfilereader(){ 
		
	}
	
	public void open(String file){
		
		try {
			close();
			
			buffer = new BufferedReader(new FileReader(file));
			
		} catch (FileNotFoundException e1) {
			e1.printStackTrace();
	
		}
		
	}
	
	public void close(){
		
		try {
			if(buffer != null){
				buffer.close();
				buffer = null;
			}
		} catch (IOException e) {
			e.printStackTrace();
		}
		
	}
	
	public String readLine() throws Exception{
		if(buffer != null){
			currentLine = buffer.readLine();
			
			if(currentLine == null)
				close();
			
			return currentLine;
		}
		else
			throw new Exception("No file to read...");
	}
	
	public String getCurrent(){
		return this.currentLine;
	}
	
	public boolean isReadable(){
		return (buffer != null && this.currentLine != null);
	}
}
    
    """
}

input_json = json.dumps(input_data_for_model)
# print(input_json)
response = requests.post(url=url, data=input_json)
print(str(response.text))

{"Output":" \n    package com.hmkcode;\n\nimport java.io.BufferedReader;\nimport java.io.FileNotFoundException;\nimport java.io.FileReader;\nimport java.io.IOException;\n\npublic class TextFileReader {\n\nprivate Bufferedreader buffer;\nprivate String CurrentLine = \"\";\n\tpublic textfilereader(){ \n\t\t\n\t}\n\t\n\tpublic void open(String file){\n\t\t\n\t\ttry {\n\t\t\tclose();\n\t\t\t\n\t\t\tbuffer = new BufferedReader(new FileReader(file));\n\t\t\t\n\t\t} catch (FileNotFoundException e1) {\n\t\t\te1.printStackTrace();\n\t\n\t\t}\n\t\t\n\t}\n\t\n\tpublic void close(){\n\t\t\n\t\ttry {\n\t\t\tif(buffer != null){\n\t\t\t\tbuffer.close();\n\t\t\t\tbuffer = null;\n\t\t\t}\n\t\t} catch (IOException e) {\n\t\t\te.printStackTrace();\n\t\t}\n\t\t\n\t}\n\t\n\tpublic String readLine() throws Exception{\n\t\tif(buffer != null){\n\t\t\tcurrentLine = buffer.readLine();\n\t\t\t\n\t\t\tif(currentLine == null)\n\t\t\t\tclose();\n\t\t\t\n\t\t\treturn currentLine;\n\t\t}\n\t\telse\n\t\t\tthrow new Ex

In [13]:
r = response.json()
print(r['Output'])

 
    package com.hmkcode;

import java.io.BufferedReader;
import java.io.FileNotFoundException;
import java.io.FileReader;
import java.io.IOException;

public class TextFileReader {

private Bufferedreader buffer;
private String CurrentLine = "";
	public textfilereader(){ 
		
	}
	
	public void open(String file){
		
		try {
			close();
			
			buffer = new BufferedReader(new FileReader(file));
			
		} catch (FileNotFoundException e1) {
			e1.printStackTrace();
	
		}
		
	}
	
	public void close(){
		
		try {
			if(buffer != null){
				buffer.close();
				buffer = null;
			}
		} catch (IOException e) {
			e.printStackTrace();
		}
		
	}
	
	public String readLine() throws Exception{
		if(buffer != null){
			currentLine = buffer.readLine();
			
			if(currentLine == null)
				close();
			
			return currentLine;
		}
		else
			throw new Exception("No file to read...");
	}
	
	public String getCurrent(){
		return this.currentLine;
	}
	
	public boolean isReadable(){
		return (buffer != null && this